In [1]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

Success !


In [2]:
# Load gridded FRP data aggregated to first observation day
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/viirs_snpp_jpss1_afd_gridstats_days')
print(f"Number of unique grid days (dissolved gridcells): {grid.size().getInfo()}")
grid = grid.select(['grid_index', 'Fire_ID', 'Ig_Date', 'Last_Date', 'max_date', 'first_obs', 'last_obs'])
# Align the grid geometries with the GridMET projection
grid = grid.map(lambda f: f.setGeometry(
    f.geometry().transform(ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
                           .first().projection())
))
print(grid.first().propertyNames().getInfo())

Number of unique grid days (dissolved gridcells): 4364
['system:index', 'grid_index', 'Fire_ID', 'max_date', 'first_obs', 'Ig_Date', 'Last_Date', 'last_obs']


In [3]:
# get a list of unique active fire days
fire_days = grid.aggregate_array('first_obs').getInfo()
fire_days = set(fire_days)
print(f"Number of unique 'first fire days': {len(fire_days)}")

# get the min and max DOY
fire_days_doy = [datetime.strptime(day, "%Y-%m-%d").timetuple().tm_yday for day in fire_days]
doy_min = min(fire_days_doy)
doy_max = max(fire_days_doy)
print(f"Fire DOY range: {doy_min} to {doy_max}")

Number of unique 'first fire days': 466
Fire DOY range: 96 to 317


In [4]:
# Load the gridmet image collection
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"\ngridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}\n")


gridMET bands available for analysis:

['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 'tmmn', 'tmmx', 'vs', 'erc', 'eto', 'bi', 'fm100', 'fm1000', 'etr', 'vpd']



In [5]:
# select our variables of interest:
# Vapor Pressure Deficit (vpd), Energy Release Component (erc), wind speed (vs)
gridmet = gridmet.select(['vpd', 'erc', 'vs'])
print(gridmet.first().bandNames().getInfo())

['vpd', 'erc', 'vs']


In [6]:
# Long-term baseline for climatology
gridmet_l = gridmet.filterDate('2008-12-31', '2023-12-31')

# load the fire bounds
fires = ee.FeatureCollection('projects/jfsp-aspen/assets/srm_fire_census_2017_to_2023_bounds')

# pre-compute the long-term climatology for each fire footprint
fire_ids = ee.List(fires.aggregate_array('Fire_ID'))

def fire_climatology(fid):
    fid = ee.String(fid)
    fire_fc = fires.filter(ee.Filter.eq('Fire_ID', fid)).first()

    # get ignition/last across all cells
    # use 14 days pre-ignition as start date
    ig_doy = ee.Date(fire_fc.get('Ig_Date')).advance(-14,'days').getRelative('day', 'year')
    ls_doy = ee.Date(fire_fc.get('Last_Date')).getRelative('day', 'year')
    subcol_fil = ee.Filter.calendarRange(ig_doy, ls_doy, 'day_of_year')
    # 15-yr mean ERC over that window
    erc_clim = gridmet_l.filter(subcol_fil).select('erc').mean()

    # mean ERC across the fire footprint
    erc_clim_fire = erc_clim.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=fire_fc.geometry(),
        scale=4000,
        tileScale=2,
        maxPixels=1e13
    ).get('erc')

    return ee.Feature(None, {
        'Fire_ID': fid,
        'erc_clim_fire': erc_clim_fire
    })

# get the results
fire_clim_fc = ee.FeatureCollection(fire_ids.map(fire_climatology))
print("Calculated the per-fire ERC 15-year average")

# Export the fire climatology table.
export_task = ee.batch.Export.table.toDrive(
    collection=fire_clim_fc,
    description='fire_clim_fc',
    fileNamePrefix='fire_clim_fc',
    fileFormat='CSV',
    folder='GRIDMET'
)

export_task.start() # Start the export task
print("\tExport to Earth Engine Asset started!")

# Pull a few features into Python to inspect
sample = fire_clim_fc.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df

Calculated the per-fire ERC 15-year average
	Export to Earth Engine Asset started!


,Fire_ID,erc_clim_fire
0,71,64.351206
1,33,72.394820
2,70,68.463011
3,17,61.911716
4,100,63.397724
5,87,58.801511
6,98,61.040806
7,113,55.427149
8,32,53.432004
9,131,59.264428


In [7]:
# generate the date pair gridmet images
def process_date_pair(pair):
    pair = ee.List(pair)
    first = ee.Date(pair.get(0))
    last  = ee.Date(pair.get(1))

    first_s = first.format('YYYY-MM-dd')
    last_s  = last.format('YYYY-MM-dd')

    # --- Compute weather summaries for the active window ---
    subcol = gridmet.filterDate(
        first.advance(-1, 'days'),
        last.advance(1, 'days')
    )
    # select the bands and calculate the statistics
    vpd_97 = subcol.select('vpd').reduce(ee.Reducer.percentile([97])).rename('vpd_97')
    erc_mean = subcol.select('erc').mean().rename('erc_mn')
    vs_max = subcol.select('vs').max().rename('vs_mx')

    # --- Combine into one image ---
    img = vpd_97.addBands([erc_mean, vs_max])

    # --- Cells for this date pair ---
    cells = grid.filter(
        ee.Filter.And(
            ee.Filter.eq('first_obs', first_s),
            ee.Filter.eq('last_obs',  last_s)
        )
    )

    # --- Reduce across all geometries in one go ---
    reduced = img.reduceRegions(
        collection=cells,
        reducer=ee.Reducer.mean(),
        scale=375,
        tileScale=6
    ).map(lambda f: f.set({
        'first_obs': first_s,
        'last_obs':  last_s
    }))

    return ee.FeatureCollection(reduced)

# --- Get unique date pairs ---
# Distinct combinations of first_obs and last_obs
unique_pairs = grid.distinct(['first_obs','last_obs'])
# Build a list of [first_obs, last_obs] pairs
date_pairs = unique_pairs.toList(unique_pairs.size()).map(
    lambda f: ee.List([
        ee.Feature(f).get('first_obs'),
        ee.Feature(f).get('last_obs')
    ])
)
print(f"Unique date pairs: {unique_pairs.size().getInfo()}")

# --- Map over them ---
results = ee.FeatureCollection(date_pairs.map(process_date_pair)).flatten()
print("Success !")

Unique date pairs: 3123
Success !


In [8]:
# extract a sample of the results and inspect
sample = results.limit(10).getInfo()
props = [f['properties'] for f in sample['features']]
df = pd.DataFrame(props)
df = df[['Fire_ID', 'first_obs', 'last_obs', 'erc_mn', 'vpd_97', 'vs_mx']]
df.head(10)

,Fire_ID,first_obs,last_obs,erc_mn,vpd_97,vs_mx
0,71,2017-05-21,2017-05-21,54.345534,0.894641,2.576301
1,71,2017-05-21,2017-05-22,56.333332,0.930000,3.200000
2,71,2017-05-21,2017-05-24,54.447571,1.161098,4.098170
3,71,2017-05-22,2017-05-22,55.500000,0.920000,3.300000
4,71,2017-05-22,2017-05-26,59.500000,1.320000,6.300000
5,71,2017-05-22,2017-05-28,60.625000,1.320000,6.300000
6,71,2017-05-24,2017-05-24,60.091356,1.237813,3.962946
7,71,2017-05-24,2017-05-26,62.713043,1.372933,6.255889
8,71,2017-05-24,2017-05-27,64.331864,1.419937,6.216719
9,71,2017-05-24,2017-05-28,62.333332,1.320000,6.300000


In [9]:
# Remove geometry
def remove_geometry(ftr):
    return ftr.setGeometry(None)
results = results.map(remove_geometry)
results = results.select([
    'Fire_ID','first_obs','last_obs','erc_mn','vpd_97','vs_mx',
])

# Export the table.
export_task = ee.batch.Export.table.toDrive(
    collection=results,
    description='gridstats_gridmet_full_v2',
    fileNamePrefix='gridstats_gridmet_full_v2',
    fileFormat='CSV',
    folder='GRIDMET_V2'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 360)

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!
